## 4.1.3 Linear Regression Example with Word2Vec

idea : word to vec

- string으로 다른 단어와 유사도를 비교를 하는 것이 아닌 의미적으로 다른 단어와 비교를 하는 것!

- 벡터화된 단어간의 유사도: jaccard, cosine

### Word2Vec Feature Example

In [1]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [4]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

#### word2vec을 활용한 모델 구현

: word2vec의 경우 단어로 표현된 리스트를 입력값으로 넣어야한다.

In [5]:
sentences = []  # 단어집합
for review in reviews:
    sentences.append(review.split())

#### word2vec 벡터화

In [6]:
# 학습에 필요한 하이퍼 파라미터
num_features = 300   # 임베딩된 벡터의 차원을 정한다, 워드 벡터 특징값 수 
min_word_count = 40  # 모델에 의미 있는 단어를 가지고 학습하기 위해 적은 빈도 수의 단어들은 학습하지 않는다, 단어에 대한 최소 빈도 수
num_workers = 4      # 모델 학습시 필요한 프로세스 개수를 지정한다.
context = 10         # 컨텍스트 윈도우 크기를 지정한다. (주변에 있는 단어를 관여, 양옆으로 10개씩)
downsampling = 1e-3  # 라벨에 대한 다운샘플링 비율을 지정한다. (전체 단어의 0.001을 어떤 단어가 넘는 순간, 빈번하게 등장한다고 판단하고 이 단어에 대해 다운샘플링을 진행하는 것으로 보임, 계산속도를 위한 feature 추상화?)

In [1]:
# !pip install gensim

In [8]:
import logging    # 로그 메시지를 양식에 맞게 info 수준으로 보기(학습과정)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

In [9]:
from gensim.models import word2vec  # 자체적으로 index생성 -> 딕셔너리(단어사전) -> 벡터

# word2vec 객체를 만든다.
model = word2vec.Word2Vec(sentences, workers=num_workers, \
                          size=num_features, min_count = min_word_count, \
                          window = context, sample = downsampling) # ephochs 조절 가능

2019-04-08 21:46:24,374 : INFO : 'pattern' package not found; tag filters are not available for English
2019-04-08 21:46:24,383 : INFO : collecting all words and their counts
2019-04-08 21:46:24,385 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-08 21:46:24,887 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2019-04-08 21:46:25,429 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2019-04-08 21:46:25,764 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2019-04-08 21:46:25,766 : INFO : Loading a fresh vocabulary
2019-04-08 21:46:25,863 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2019-04-08 21:46:25,865 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2019-04-08 21:46:25,908 : INFO : deleting the raw counts dictionary of 74065 items


In [10]:
# 모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있다.
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-04-08 21:49:19,730 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2019-04-08 21:49:19,732 : INFO : not storing attribute vectors_norm
2019-04-08 21:49:19,733 : INFO : not storing attribute cum_table
2019-04-08 21:49:20,082 : INFO : saved 300features_40minwords_10context


In [11]:
# 리뷰마다 단어의 개수가 모두 다르기 때문에 입력값을 하나의 형태로 만들어야 한다.
# 문장에 있는 모든 단어의 벡터값에 대해 평균을 내서 리뷰 하나당 하나의 벡터로 만드는 방법
# -> 단어의 정보를 살려서 붙여서 사용하는 방법을 사용하는 경우도 많음
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    # 어휘사전 준비
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w]) # model: word2vec 모델
    
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words) # 임베딩하는 차원 수
    
    return feature_vector

In [12]:
# 각 리뷰의 평균 벡터를 구하는 함수
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))  # 리뷰 하나당 하나의 벡터

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [13]:
train_data_vecs = get_dataset(sentences, model, num_features)

c:\users\nsa18\anaconda3\envs\ml_nlp\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [14]:
train_data_vecs

array([[-0.25949094, -0.02115187, -0.32396752, ...,  0.21128088,
         0.18315448, -0.2859757 ],
       [ 0.14265062,  0.06817823,  0.09425162, ...,  0.26768705,
         0.30138168, -0.1131453 ],
       [ 0.08122231, -0.12266844, -0.03936379, ...,  0.07450753,
         0.07171711, -0.02665397],
       ...,
       [-0.28711239, -0.07738554, -0.43110138, ...,  0.22086881,
         0.19231258, -0.15484633],
       [-0.08024016,  0.14691758, -0.1669161 , ...,  0.23371397,
         0.3719498 , -0.15348841],
       [-0.32940215,  0.19108546, -0.02552948, ...,  0.09473239,
         0.23248293, -0.36190802]], dtype=float32)

#### 학습과 검증 데이터셋 분리

In [15]:
from sklearn.model_selection import train_test_split
import numpy as np

X = train_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

#### 모델 선언 및 학습

In [16]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

c:\users\nsa18\anaconda3\envs\ml_nlp\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

#### 검증 데이터셋을 이용한 성능 평가

In [17]:
predicted = lgs.predict(X_test)
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

# word2vec의 경우도 항상 좋은 결과를 만들지는 않는다.
print("------------")
print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

------------
Accuracy: 0.862800
Precision: 0.857868
Recall: 0.872171
F1-Score: 0.864961
AUC: 0.934065


#### 데이터 제출

In [18]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])

In [19]:
test_data.head(5)

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


In [20]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

In [21]:
test_data_vecs = get_dataset(test_sentences, model, num_features)  # 임베딩된 벡터값을 갖게한다.

c:\users\nsa18\anaconda3\envs\ml_nlp\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [22]:
test_predicted = lgs.predict(test_data_vecs)

In [23]:
ids = list(test_data['id'])

answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})

In [24]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)